## 1. How many Spam Emails are there?

In [1]:
def count_spam(path = "D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data", folders = ['enron1','enron2']):
    import os
    from pathlib import Path
    #path = Path("D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data")
    path = Path(path)
    spam_count = 0
    for folder in folders:
        file_names = os.listdir(path / folder / 'spam')
        for file_name in file_names:
            if 'spam' in file_name:
                spam_count += 1
    print('There are', spam_count, 'spam emails in', folders)

In [2]:
count_spam()

There are 2993 spam emails in ['enron1', 'enron2']


## 2. Structure the email data from the 2 directories into 1 dataframe with columns: Status, Subject, Body.

#### Specify the folders that will be included and the path leading to the folders.

In [3]:
folders = ['enron1','enron2']
status = ['spam','ham']
path1 = "D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\{folder}\\{status}"

#### Create the paths containing all the desired emails.

In [4]:
from itertools import product
directories = list(product(folders, status))
directories

[('enron1', 'spam'), ('enron1', 'ham'), ('enron2', 'spam'), ('enron2', 'ham')]

In [5]:
paths = []
for i in directories:
    paths.append(path1.format(folder=i[0],status=i[1]))
paths

['D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron2\\spam',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron2\\ham']

#### List all the files in each `path`.

In [6]:
import glob
file_paths = [glob.glob(pathname=path + '\\*') for path in paths]

#### Now we have all the file paths of the emails in the specified folders.

In [7]:
file_paths[0][0:5]

['D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam\\0006.2003-12-18.GP.spam.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam\\0008.2003-12-18.GP.spam.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam\\0017.2003-12-18.GP.spam.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam\\0018.2003-12-18.GP.spam.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam\\0026.2003-12-18.GP.spam.txt']

In [8]:
file_paths[1][0:5]

['D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham\\0001.1999-12-10.farmer.ham.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham\\0002.1999-12-13.farmer.ham.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham\\0003.1999-12-14.farmer.ham.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham\\0004.1999-12-14.farmer.ham.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham\\0005.1999-12-14.farmer.ham.txt']

#### Having acquired the file paths, we can now extract the status, subject and message body of the emails.

In [9]:
subjects = []
bodies = []
status = []

# for each path
for file_path in file_paths:
    # for each file in path
    for i in file_path:
        # for status
        if 'spam.txt' in i:
            status.append('spam')
        elif 'ham.txt' in i:
            status.append('ham')
        # extract the subject from email
        with open(i,'r',errors='ignore') as f:
            subject = f.readlines()[0]
            subject = subject[9:]
            subjects.append(subject)
        # extract body message from email
        with open(i,'r',errors='ignore') as f:
            body = f.readlines()
            body = body[1:]
            bodies.append(body)
            
# join each sub-elements of the bodies list
bodies = [' '.join(i) for i in bodies]

print('Status has', len(status), 'elements.')
print('Subject has', len(subjects), 'elements.')
print('Body has', len(bodies), 'elements.')

Status has 11026 elements.
Subject has 11026 elements.
Body has 11026 elements.


#### Now, use the lists to create a dataframe.

In [10]:
import pandas as pd

In [11]:
enron = pd.DataFrame(list(zip(status,subjects,bodies)), columns=['Status','Subject','Body'])
enron.sample(10)

,Status,Subject,Body
6409,spam,use this handy interest calculator to get curr...,use this handy interest calculator to get curr...
3890,ham,new computer\n,hey ! just checking out the new machine . i se...
7715,ham,mid - year 2000 performance feedback\n,note : you will receive this message each time...
2283,ham,here ' s my performance evaluation choices :\n,gary hanks - pipeline operations\n morgan shar...
7121,ham,bonds\n,"hello , vince .\n we still have the 2 other bo..."
7159,ham,welcome\n,network world fusion focus : jason meserve on\...
5184,spam,add logos and tones to your cell phone 1575332...,take yourself out of our list by clicking here...
2933,ham,feedback monitor error - meter 984132 - 1 / 99\n,- - - - - - - - - - - - - - - - - - - - - - fo...
7933,ham,re : pre - meeting weathereffects site cruise\n,"vince ,\n you ' re right . it is wednesday ! s..."
5173,spam,"urgent business proposal ,\n",mrs . regina rossman .\n # 263 sandton city\n ...


## 3. Build a Naive Bayes classifier to classify whether emails are spam or not.

#### Import libraries

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [13]:
enron.sample(5)

,Status,Subject,Body
1507,ham,king ranch\n,there are two fields of gas that i am having d...
5001,ham,"ponderosa pine energy partners , ltd . - gas\n",brazos is not requesting any gas volumes throu...
8091,ham,re : american gas association\n,"mike ,\n please , draft a message to js , dd a..."
7889,ham,telephone interview with the research group\n,good morning mr . catanese :\n your resume was...
5886,spam,can people find your web site ?\n,"expedite 245 west roosevelt rd . building 15 ,..."


In [14]:
enron.shape

(11026, 3)

#### Create `Spam` column from the `Status` column and delete `Status` column.

In [15]:
enron['Spam'] = [1 if i == 'spam' else 0 for i in enron['Status']]
enron.drop('Status', inplace=True, axis=1)

In [16]:
enron.sample(5)

,Subject,Body,Spam
7763,volumetric optionality model changes\n,- - - - - - - - - - - - - - - - - - - - - - fo...,0
10289,erisk essentials\n,? www . erisk . com\n what ' s new at erisk . ...,0
3947,eastrans nomination effective 1 / 25 / 01\n,please increase deliveries into eastrans to 25...,0
2082,updated buyback worksheet\n,please see the attached . the undertake for ai...,0
9911,wharton interviews\n,- - - - - - - - - - - - - - - - - - - - - - fo...,0


#### Split data for training and testing.

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train_X, test_X, train_y, test_y = train_test_split(enron["Body"], enron["Spam"], test_size=0.3, random_state=0)

#### Import libraries for text analysis

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB

In [20]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_train_X = vectorizer.fit_transform(train_X)

In [21]:
# index_value={i[1]:i[0] for i in vectorizer.vocabulary_.items()}

In [22]:
# fully_indexed = []
# for row in tfidf_train_X:
#     fully_indexed.append({index_value[column]:value for (column,value) in zip(row.indices,row.data)})

#### Train the model

In [23]:
classifier = GaussianNB()
classifier.fit(tfidf_train_X.toarray(), train_y)

GaussianNB()

## 4. What is the longest ham email?

#### Create a dataframe containing all ham emails.

In [24]:
hams = enron[enron['Spam'] == 0][['Body']]

#### Create a column containing the lengths of each emails.

In [25]:
hams['Lengths'] = [len(i) for i in hams['Body']]
hams.drop('Body',inplace=True,axis=1)

#### Get the index of the longest ham email.

In [26]:
longest_ham = hams.idxmax()
print(longest_ham)

Lengths    10765
dtype: int64


#### Print the longest ham email.

In [27]:
enron.iloc[longest_ham,:].values

array([['from the enron india newsdesk - april 27 th newsclips\n',
        'fyi news articles from indian press .\n - - - - - - - - - - - - - - - - - - - - - - forwarded by sandeep kohli / enron _ development on 04 / 27 / 2001 08 : 24 am - - - - - - - - - - - - - - - - - - - - - - - - - - -\n nikita varma\n 04 / 27 / 2001 07 : 51 am\n to : nikita varma / enron _ development @ enron _ development\n cc : ( bcc : sandeep kohli / enron _ development )\n subject : from the enron india newsdesk - april 27 th newsclips\n friday apr 27 2001 , http : / / www . economictimes . com / today / cmo 3 . htm\n dpc board empowers md to cancel mseb contract\n friday apr 27 2001 , http : / / www . economictimes . com / today / 27 compl 1 . htm\n mseb pays rs 134 cr under \' protest \' to dpc\n friday , april 27 , 001 , http : / / www . businessstandard . com / today / economy 4 . asp ? menu = 3\n enron india md authorised to terminate ppa\n friday , april 27 , 2001 , http : / / www . financialexpress . c

#### The longest ham email has `43888` characters.

In [28]:
hams.max()

Lengths    43888
dtype: int64

## 5. What is the accuracy of your model?

In [29]:
from sklearn.model_selection import cross_val_score

In [30]:
tfidf_test_X = vectorizer.transform(test_X)
print(tfidf_test_X.shape)
scores = cross_val_score(classifier, tfidf_test_X.toarray(), test_y, cv=5)
acc = scores.mean()
print("Accuracy: %0.2f percent" % (acc *100))

(3308, 58021)
Accuracy: 94.01 percent


**The model has an accuracy of `94.01%`.**

## 6. Include the Subject in the analysis of the emails, does the accuracy/performance of the model increase?

#### Create a new column containing the concatenation of `Subject` and `Body`.

In [31]:
enron['Total'] = [sub + bod for (sub, bod) in zip(enron.iloc[:,0],enron.iloc[:,1])]

In [32]:
enron.head(5)

,Subject,Body,Spam,Total
0,dobmeos with hgh my energy level has gone up !...,introducing\n doctor - formulated\n hgh\n huma...,1,dobmeos with hgh my energy level has gone up !...
1,your prescription is ready . . oxwq s f e\n,"low cost prescription medications\n soma , ult...",1,your prescription is ready . . oxwq s f e\nlow...
2,get that new car 8434\n,people nowthe weather or climate in any partic...,1,get that new car 8434\npeople nowthe weather o...
3,await your response\n,"dear partner ,\n we are a team of government o...",1,"await your response\ndear partner ,\n we are a..."
4,"coca cola , mbna america , nascar partner with...",stock\n profile\n about\n company\n investment...,1,"coca cola , mbna america , nascar partner with..."


In [33]:
train_X, test_X, train_y, test_y = train_test_split(enron["Total"], enron["Spam"], test_size=0.3, random_state=0)

In [34]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_train_X = vectorizer.fit_transform(train_X)

In [35]:
# index_value={i[1]:i[0] for i in vectorizer.vocabulary_.items()}

In [36]:
# fully_indexed = []
# for row in tfidf_train_X:
#     fully_indexed.append({index_value[column]:value for (column,value) in zip(row.indices,row.data)})

In [37]:
classifier = GaussianNB()
classifier.fit(tfidf_train_X.toarray(), train_y)

GaussianNB()

In [38]:
tfidf_test_X = vectorizer.transform(test_X)
print(tfidf_test_X.shape)
scores = cross_val_score(classifier, tfidf_test_X.toarray(), test_y, cv=5)
acc = scores.mean()
print("Accuracy: %0.2f percent" % (acc *100))

(3308, 58864)
Accuracy: 94.68 percent


**The accuracy improved by `0.67%`**

## Bonus : Answer questions 1-6 using Enron 1,2, and 3.

**Answer to bonus question is in the jupyter notebook `PS3 - Bonus - Jake.ipynb`.**